# Email Bot Notebook

This notebook demonstrates how to:
1. Authenticate with Gmail's API.
2. Retrieve unread emails.
3. Summarize or filter them (optionally).
4. Send a daily recap email back to yourself.
5. Mark processed emails as read.

We'll use the [Google API Python Client](https://github.com/googleapis/google-api-python-client) for Gmail access.


## 1. Setup and Installation

Make sure to install the necessary packages in your environment:

In [ ]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

## 2. Imports and Configuration

We'll import the libraries we need and set up any global constants such as:
- `SCOPES`: Gmail API scopes
- `CREDENTIALS_PATH`: Path to our `credentials.json`
- `TOKEN_PATH`: Path to store the OAuth tokens (so we don’t have to re-authenticate every time).


In [ ]:
import os
import base64
import datetime

from email.mime.text import MIMEText

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# If modifying these scopes, delete your existing token file.
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

# Paths to your files
CREDENTIALS_PATH = "secrets/credentials.json"  # Adjust as needed
TOKEN_PATH = "secrets/token.pickle"  # Where the OAuth tokens will be stored

## 3. Authentication

We'll do the following:
1. Check if we already have a valid `token.pickle`.
2. If not, we prompt the user to log in via OAuth.
3. We then save the token for future sessions.


In [ ]:
def get_gmail_service():
    creds = None

    # 1. Check if token.pickle exists
    if os.path.exists(TOKEN_PATH):
        with open(TOKEN_PATH, "rb") as token_file:
            creds = pickle.load(token_file)

    # 2. If no valid creds or they are expired, go through the OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
            creds = flow.run_local_server(port=0)

        # 3. Save the credentials
        with open(TOKEN_PATH, "wb") as token_file:
            pickle.dump(creds, token_file)

    # 4. Build the Gmail service
    service = build("gmail", "v1", credentials=creds)
    return service


# Test it once
service = get_gmail_service()
print("Gmail service created successfully.")

## 4. Retrieving Unread Emails

We'll fetch a list of messages labeled `UNREAD`, retrieve their details, and store them for processing.


In [ ]:
def get_unread_emails(service, max_results=10):
    """
    Fetch a list of unread email message IDs,
    then retrieve their data (subject, snippet, etc.).
    """
    try:
        # Search for unread messages
        results = (
            service.users()
            .messages()
            .list(userId="me", labelIds=["UNREAD"], maxResults=max_results)
            .execute()
        )

        messages = results.get("messages", [])

        emails_data = []
        for msg in messages:
            # Get the message details
            msg_detail = (
                service.users().messages().get(userId="me", id=msg["id"]).execute()
            )

            subject = None
            snippet = msg_detail.get("snippet", "")

            # Parse headers for Subject
            headers = msg_detail.get("payload", {}).get("headers", [])
            for header in headers:
                if header["name"].lower() == "subject":
                    subject = header["value"]
                    break

            emails_data.append(
                {"id": msg["id"], "subject": subject, "snippet": snippet}
            )

        return emails_data

    except HttpError as error:
        print(f"An error occurred: {error}")
        return []


unread_emails = get_unread_emails(service)
print("Unread Emails:", unread_emails)

## 5. Summarizing or Filtering Emails (Optional)

You can summarize emails using an AI API (like OpenAI's GPT-3.5/4) or by simple keyword matching. For now, we'll just do a naive summary or identify important emails based on some condition (e.g., subject keywords).


In [ ]:
def summarize_or_filter(emails):
    """
    Placeholder: apply some logic or AI to summarize
    or choose which emails are important.
    Here, we'll label any subject with 'Invoice' or 'Urgent' as important.
    """
    important = []
    unimportant = []

    for email in emails:
        subject_lower = (email["subject"] or "").lower()
        if "invoice" in subject_lower or "urgent" in subject_lower:
            important.append(email)
        else:
            unimportant.append(email)

    return important, unimportant


important_emails, other_emails = summarize_or_filter(unread_emails)
print("Important Emails:", important_emails)
print("Other Emails:", other_emails)

## 6. Sending a Daily Recap Email

We'll compose a summary of important emails and send it back to ourselves.


In [ ]:
def create_message(to, subject, body_text):
    """
    Create a message for an email.
    """
    message = MIMEText(body_text)
    message["to"] = to
    message["subject"] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes())
    return {"raw": raw.decode()}


def send_message(service, user_id, message):
    """
    Send an email message via Gmail API.
    """
    try:
        sent = service.users().messages().send(userId=user_id, body=message).execute()
        print(f"Message sent: {sent['id']}")
        return sent
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None


def send_daily_recap(service, important, unimportant):
    recap_subject = f"Daily Recap - {datetime.date.today()}"

    # Build a quick text summary
    body_lines = []
    if important:
        body_lines.append("**Important Emails:**")
        for e in important:
            body_lines.append(f"- {e['subject']} (ID: {e['id']})")
        body_lines.append("")
    else:
        body_lines.append("No important emails today.\n")

    if unimportant:
        body_lines.append("Other Unread Emails:")
        for e in unimportant:
            body_lines.append(f"- {e['subject']} (ID: {e['id']})")
    else:
        body_lines.append("No other unread emails.")

    recap_body = "\n".join(body_lines)

    # Create and send
    msg = create_message("me", recap_subject, recap_body)
    send_message(service, "me", msg)


# Example usage
send_daily_recap(service, important_emails, other_emails)

## 7. Marking Emails as Read

After processing, we can label the emails with `READ` so they won’t appear as unread in your inbox. This step is especially useful if you don’t plan to keep them for later review in unread form.


In [ ]:
def mark_emails_as_read(service, emails):
    """
    Mark emails as read in Gmail.
    """
    for email in emails:
        try:
            service.users().messages().modify(
                userId="me", id=email["id"], body={"removeLabelIds": ["UNREAD"]}
            ).execute()
        except HttpError as error:
            print(f"Error marking email as read: {error}")
            continue


# Mark everything as read (important and other) in this example
mark_emails_as_read(service, important_emails + other_emails)
print("All processed emails have been marked as read.")

# Conclusion

- We authenticated with Gmail's API.
- Retrieved unread messages.
- Applied a (basic) filter for important/unimportant emails.
- Sent ourselves a daily recap.
- Marked the emails as read.

**Next Steps**:
- Improve the filtering or summarizing with AI (OpenAI GPT, local LLM, etc.).
- Handle attachments or other advanced email features.
- Schedule this script (via `launchd` or `cron`) to run automatically each day.
